In [1]:
import pandas as pd
import numpy as np

RAW_recipes = pd.read_csv('RAW_recipes.csv')
RAW_interactions = pd.read_csv('RAW_interactions.csv')

In [2]:
RAW_recipes

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
5,apple a day milk shake,5289,0,1533,1999-12-06,"['15-minutes-or-less', 'time-to-make', 'course...","[160.2, 10.0, 55.0, 3.0, 9.0, 20.0, 7.0]",4,"['combine ingredients in blender', 'cover and ...",NaN,"['milk', 'vanilla ice cream', 'frozen apple ju...",4
6,aww marinated olives,25274,15,21730,2002-04-14,"['15-minutes-or-less', 'time-to-make', 'course...","[380.7, 53.0, 7.0, 24.0, 6.0, 24.0, 6.0]",4,['toast the fennel seeds and lightly crush the...,my italian mil was thoroughly impressed by my ...,"['fennel seeds', 'green olives', 'ripe olives'...",9
7,backyard style barbecued ribs,67888,120,10404,2003-07-30,"['weeknight', 'time-to-make', 'course', 'main-...","[1109.5, 83.0, 378.0, 275.0, 96.0, 86.0, 36.0]",10,['in a medium saucepan combine all the ingredi...,this recipe is posted by request and was origi...,"['pork spareribs', 'soy sauce', 'fresh garlic'...",22
8,bananas 4 ice cream pie,70971,180,102353,2003-09-10,"['weeknight', 'time-to-make', 'course', 'main-...","[4270.8, 254.0, 1306.0, 111.0, 127.0, 431.0, 2...",8,"['crumble cookies into a 9-inch pie plate , or...",NaN,"['chocolate sandwich style cookies', 'chocolat...",6
9,beat this banana bread,75452,70,15892,2003-11-04,"['weeknight', 'time-to-make', 'course', 'main-...","[2669.3, 160.0, 976.0, 107.0, 62.0, 310.0, 138.0]",12,"['preheat oven to 350 degrees', 'butter two 9x...",from ann hodgman's,"['sugar', 'unsalted butter', 'bananas', 'eggs'...",9


In [ ]:
RAW_interactions

In [ ]:
idx2recipe = [RAW_recipes.loc[i, 'id'] for i in range(len(RAW_recipes))]
recipe2idx = {idx2recipe[i]:i for i in range(len(idx2recipe))}
idx2user = list(set([RAW_interactions.loc[i, 'user_id'] for i in range(len(RAW_interactions))]))
user2idx = {idx2user[i]:i for i in range(len(idx2user))}

In [ ]:
ingred2idx = {}
idx2ingred = []
recipes = []
ingreds = []
for i in range(len(RAW_recipes)):
    s = RAW_recipes.loc[i, 'ingredients'].replace('[', '').replace(']', '').replace(',', '').replace("'", '').split(" ")
    for ingred in s:
        if len(ingred) > 0 and not ingred.isspace(): 
            if ingred not in ingred2idx:
                ingred2idx[ingred] = len(ingred2idx)
                idx2ingred.append(ingred)
            recipes.append(recipe2idx[RAW_interactions.loc[i, 'recipe_id']])
            ingreds.append(ingred2idx[ingred])
len(idx2ingred)

In [ ]:
users = [user2idx[RAW_interactions.loc[i, 'user_id']] for i in range(len(RAW_interactions)) if RAW_interactions.loc[i, 'rating'] == 5]
recipes = [recipe2idx[RAW_interactions.loc[i, 'recipe_id']] for i in range(len(RAW_interactions)) if RAW_interactions.loc[i, 'rating'] == 5]

In [ ]:
data = [1 for _ in range(len(recipes))]

In [ ]:
print(len(ingreds), len(recipes), len(data), len(ingred2idx), len(recipe2idx))

In [ ]:
from scipy.sparse import csr_matrix
matrix = csr_matrix((data, (ingreds, recipes)), shape=(len(ingred2idx), len(recipe2idx)))

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, n_iter=200, random_state=42)
svd.fit(matrix)
transformed = svd.transform(matrix)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components= 4, random_state=0)
lda.fit(matrix)
transformed = lda.transform(matrix)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(transformed)
clusters = kmeans.transform(transformed)

In [ ]:
ind = np.argpartition(transformed[:, 3], -20)[-20:]
[idx2ingred[i] for i in ind]